In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
customers_schema = "customer_id integer,tax_id STRING,tax_code STRING, customer_name STRING,state STRING,city STRING,postcode STRING,street STRING,number STRING,unit STRING,region STRING,district STRING,lon double,lat double,ship_to_address STRING,valid_from STRING,valid_to STRING,units_purchased STRING,loyalty_segment integer, customer_key BIGINT"

customers_df = spark.read \
        .format("delta") \
        .load("dbfs:/user/hive/warehouse/example.db/customers_cleansed_dt")

customers_df = customers_df.withColumn("customer_key", expr("monotonically_increasing_id() + 1"))

dim_customers = spark.createDataFrame(customers_df.rdd, customers_schema)

dim_customers.write.format("delta").saveAsTable("example.dim_customers_dt")

In [ ]:
products_schema = "product_id STRING,product_category STRING,product_name STRING,sales_price STRING,ean13 DOUBLE,ean5 STRING,product_unit STRING, product_key BIGINT"

products_df = spark.read \
        .format("delta") \
        .load("dbfs:/user/hive/warehouse/example.db/products_cleansed_dt") \
        .select("product_id", "product_category", "product_name", "sales_price", "ean13", "ean5", "product_unit")

products_df = products_df.withColumn("product_key", expr("monotonically_increasing_id() + 1"))

dim_products = spark.createDataFrame(products_df.rdd, products_schema)

dim_products.write.format("delta").saveAsTable("example.dim_products_dt")